In [7]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision

from torch_geometric.datasets import QM9
import torch_geometric.transforms as T
import torch
from torch_geometric.loader import DataLoader
from data_utils import *
import random
import os

from data_utils import create_tensorboard_writer

# disable logging
from rdkit import RDLogger
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = T.Compose([
    SelectQM9TargetProperties(properties=["homo", "lumo"]),
    SelectQM9NodeFeatures(features=["atom_type"]),
    T.ToDevice(device=device)
])

dataset = QM9(root="./data", transform=transform)

train_dataset, _, _ = create_qm9_data_split(dataset=dataset)

sample_index = random.randint(0, len(train_dataset))
print(sample_index)
sample_index = 99583
sample = train_dataset[sample_index]

# TODO: split the batch into subgraphs and display with make_grid from torchvision
#sample = Data(x=batch.x, edge_index=batch.edge_index, edge_attr=batch.edge_attr)
writer = create_tensorboard_writer(experiment_name="qm9-visualization")

print(f"SMILE: {sample.smiles}")
writer.add_image('Molecules (from SMILES)', smiles_to_image(sample.smiles), dataformats="NCHW")
writer.add_image('Molecules (from Graph)', molecule_graph_data_to_image(sample), dataformats="NCHW")
writer.close()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
53155
SMILE: [H]O[C@@]1([H])C([H])([H])[C@@]2([H])C(=O)[C@]1([H])N2[H]
